# Single Subject Segmentation Q1K

In [1]:
# import packages
import q1k_sync_tools as qst
import mne
mne.viz.set_browser_backend('qt')
%matplotlib qt
import pandas as pd
pd.options.mode.chained_assignment = None
import pylossless as ll
import numpy as np
import glob

import matplotlib.pyplot as plt

from mne.time_frequency import tfr_morlet
from mne.stats import permutation_cluster_test
from mne.datasets import sample

<frozen importlib._bootstrap>:219: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted


Using qt as 2D backend.


In [2]:
# set file variables
#subject_raw_eeg = glob.glob('sourcedata/020_1/020_1_go_*.mff')
subject_raw_eeg = 'sourcedata/024_1/024_1_eeg/024_1_ap_20230710_110505.mff'
subject_raw_et = 'sourcedata/024_1/024_1_eyetracking/024_ap_1.asc'
subject_derivative = 'output_ap/derivatives/pylossless/sub-024/eeg/sub-024_task-ap_eeg.edf'
task_code = 'ap'

In [3]:
# Load EEG recording
eeg_raw = mne.io.read_raw_egi(subject_raw_eeg)

# Get event info from source recording
eeg_events = mne.find_events(eeg_raw, shortest_event=1)
eeg_event_dict = qst.get_event_dict(eeg_raw, eeg_events)

Reading EGI MFF Header from /home/james/q1k/pilot/q1k-external-pilot/sourcedata/024_1/024_1_eeg/024_1_ap_20230710_110505.mff...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {} ...


/tmp/ipykernel_13394/896616945.py:5: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted
  eeg_events = mne.find_events(eeg_raw, shortest_event=1)


6129 events found
Event IDs: [1 2 3 4 5 6]


In [ ]:
eeg_events

In [ ]:
eeg_event_dict

In [ ]:
# Plot channel data only
mne.viz.plot_raw(eeg_raw, events=eeg_events, event_id=eeg_event_dict)

In [4]:
# Build initial state; nothing applied yet
ll_state = ll.LosslessPipeline()
ll_state = ll_state.load_ll_derivative(subject_derivative)

Extracting EDF parameters from /home/james/q1k/pilot/q1k-external-pilot/output_ap/derivatives/pylossless/sub-024/eeg/sub-024_task-ap_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from output_ap/derivatives/pylossless/sub-024/eeg/sub-024_task-ap_events.tsv.
Reading channel info from output_ap/derivatives/pylossless/sub-024/eeg/sub-024_task-ap_channels.tsv.
Reading electrode coords from output_ap/derivatives/pylossless/sub-024/eeg/sub-024_space-CapTrak_electrodes.tsv.
Reading output_ap/derivatives/pylossless/sub-024/eeg/sub-024_task-ap_ica1_ica.fif ...
Now restoring ICA solution ...
Ready.
Reading output_ap/derivatives/pylossless/sub-024/eeg/sub-024_task-ap_ica2_ica.fif ...
Now restoring ICA solution ...
Ready.


/home/james/q1k/pilot/q1k-external-pilot/eeg-env/lib/python3.8/site-packages/pylossless/pipeline.py:1118: RuntimeWarning: The number of channels in the channels.tsv sidecar file (136) does not match the number of channels in the raw data file (129). Will not try to set channel names.
  self.raw = mne_bids.read_raw_bids(derivatives_path)
/home/james/q1k/pilot/q1k-external-pilot/eeg-env/lib/python3.8/site-packages/pylossless/pipeline.py:1118: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: DIN4, STI 014, TSYN, VBeg, ae06, ae40, dbrk
  self.raw = mne_bids.read_raw_bids(derivatives_path)


In [ ]:
# Plot channel data only
mne.viz.plot_raw(ll_state.raw, events=eeg_events, event_id=eeg_event_dict)

In [ ]:
# Plot comonents
ll_state.ica2.plot_sources(ll_state.raw)


In [ ]:
# Plot topos
ll_state.ica2.plot_components(inst=ll_state.raw)

In [5]:
# Merge marks down to bads (aka manual)
ll_qcr = ll_state.raw.copy()
manual = []
for flag_type in ll_state.flags['ch']:
    manual.extend(ll_state.flags['ch'][flag_type])
print(ll_state.flags['ch'])
ll_qcr.info['bads'].extend(manual)
ll_qcr.info['bads'] = list(set(ll_qcr.info['bads']))

# Read the ICLabel info from file and add to exclude
df = pd.read_csv(subject_derivative.replace('_eeg.edf', '_iclabels.tsv'), sep='\t')
ll_state.ica2.exclude = list(df[df['ic_type'].str.match('eog|muscle|ch_noise|ecg')].index)

# Load the data and apply the ICA
ll_qcr.load_data()
ll_state.ica2.apply(ll_qcr)
ll_qcr = ll_qcr.filter(l_freq=1.0, h_freq=90.0)
ll_qcr = ll_qcr.set_eeg_reference(ref_channels="average")
ll_qcr

{'bridge': array(['E3', 'E4', 'E80', 'E89', 'E94', 'E117', 'E118', 'E123', 'E124'],
      dtype=object), 'ch_sd': array(['E14', 'E17', 'E21', 'E73', 'E88'], dtype=object), 'rank': array(['E24'], dtype=object)}
Reading 0 ... 396999  =      0.000 ...   396.999 secs...
Applying ICA to Raw instance
    Transforming to ICA space (109 components)
    Zeroing out 17 ICA components
    Projecting back using 110 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 90 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 90.00 Hz
- Upper transition bandwidth: 22.50 Hz (-6 dB cutoff frequency: 101.25 Hz)
- Filter length: 3301 samples (3.301 s)



/tmp/ipykernel_13394/2504360066.py:17: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted
  ll_qcr = ll_qcr.filter(l_freq=1.0, h_freq=90.0)
/tmp/ipykernel_13394/2504360066.py:17: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted
  ll_qcr = ll_qcr.filter(l_freq=1.0, h_freq=90.0)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


[Parallel(n_jobs=1)]: Done 129 out of 129 | elapsed:    2.5s finished
/home/james/q1k/pilot/q1k-external-pilot/eeg-env/lib/python3.8/site-packages/IPython/core/formatters.py:344: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted
  return method()
/home/james/q1k/pilot/q1k-external-pilot/eeg-env/lib/python3.8/site-packages/IPython/core/formatters.py:344: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted
  return method()


<RawEDF | sub-024_task-ap_eeg.edf, 129 x 397000 (397.0 s), ~390.9 MB, data loaded>

In [ ]:
# Plot cleaned data
mne.viz.plot_raw(ll_qcr, events=eeg_events, event_id=eeg_event_dict)

In [7]:
# Take a look at the event dictionairy
eeg_event_dict

{'ae40': 1, 'ae06': 2, 'TSYN': 3, 'dbrk': 4, 'DIN4': 5, 'VBeg': 6}

In [7]:
# Relabel condition vars for niceness
#eeg_event_dict['fq06hz/target/disp/ae06'] = eeg_event_dict.pop('ae06')
#eeg_event_dict['fq40hz/target/disp/ae40'] = eeg_event_dict.pop('ae40')

In [8]:
event_id = 1
tmin, tmax = -1, 2

epochs_condition_40hz = mne.Epochs(
    ll_qcr,
    eeg_events,
    event_id,
    tmin,
    tmax,
    baseline=(None, 0),
    preload=True,
)

ch_name = 'E6'
epochs_condition_40hz.pick_channels([ch_name])

Not setting metadata
80 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 80 events and 3001 original time points ...


2 bad epochs dropped
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/home/james/q1k/pilot/q1k-external-pilot/eeg-env/lib/python3.8/site-packages/IPython/core/formatters.py:344: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted
  return method()


Number of events,78
Events,1: 78
Time range,-1.000 – 2.000 s
Baseline,-1.000 – 0.000 s


In [9]:
decim = 2
freqs = np.arange(30, 90, 2)  # define frequencies of interest
n_cycles = freqs / 2

pow_1, itc_1 = tfr_morlet(
    epochs_condition_40hz,
    freqs,
    n_cycles=n_cycles,
    decim=decim,
    return_itc=True,
    average=True,
)

#pow_2, itc_2 = tfr_morlet(
#    epochs_condition_2,
#    freqs,
#    n_cycles=n_cycles,
#   decim=decim,
#    return_itc=True,
#    average=True,
#)

#tfr_epochs_1.apply_baseline(mode="ratio", baseline=(None, 0))
#tfr_epochs_2.apply_baseline(mode="ratio", baseline=(None, 0))

itc_dat_1 = itc_1.data[0, :, :]  # only 1 channel as 3D matrix
pow_dat_1 = pow_1.data[0, :, :]  # only 1 channel as 3D matrix

/tmp/ipykernel_13394/1775250285.py:5: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted
  pow_1, itc_1 = tfr_morlet(
/tmp/ipykernel_13394/1775250285.py:5: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted
  pow_1, itc_1 = tfr_morlet(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s finished


In [8]:
itc_1.data.shape

(1, 23, 1501)

In [10]:
times = 1e3 * epochs_condition_40hz.times  # change unit to ms

fig, (ax, ax2) = plt.subplots(2, 1, figsize=(6, 4))
fig.subplots_adjust(0.12, 0.08, 0.96, 0.94, 0.2, 0.43)

ax.imshow(
    pow_dat_1,
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
    aspect="auto",
    origin="lower",
    cmap="RdBu_r",
)

ax2.imshow(
    itc_dat_1,
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
    aspect="auto",
    origin="lower",
    cmap="RdBu_r",
)

ax.set_xlabel("Time (ms)")
ax.set_ylabel("Frequency (Hz)")
ax.set_title(f"Induced power ({ch_name})")

Text(0.5, 1.0, 'Induced power (E6)')

In [9]:
# Epoch structure is created for ALL events, then you 'segment' by indexing into it
epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1, tmax=2.0, on_missing='warn', event_repeated='drop')

Not setting metadata
1331 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated


/tmp/ipykernel_6495/3552720110.py:2: RuntimeWarning: No matching events found for VBeg (event id 13)
  epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1, tmax=2.0, on_missing='warn', event_repeated='drop')


In [10]:
# Check trial counts
epochs[['fq06hz', 'fq15hz']]

/home/james/q1k/pilot/q1k-external-pilot/eeg-env/lib/python3.8/site-packages/IPython/core/formatters.py:344: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted
  return method()


Number of events,120
Events,fq06hz/target/disp/dstr: 60fq15hz/target/disp/fvct: 60
Time range,-1.000 – 2.000 s
Baseline,-1.000 – 0.000 s


In [11]:
# Plot ERPs
evokeds = {'fq06hz': epochs['fq06hz'].average(), 'fq15hz': epochs['fq15hz'].average()}
mne.viz.plot_compare_evokeds(evokeds, picks=['E72'], combine='mean')

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
combining channels using "mean"
combining channels using "mean"


/tmp/ipykernel_6495/3322987310.py:3: RuntimeWarning: Only 1 channel in "picks"; cannot combine by method "mean".
  mne.viz.plot_compare_evokeds(evokeds, picks=['E72'], combine='mean')


[<Figure size 800x600 with 2 Axes>]

In [12]:
mne.write_evokeds(f'erp_fif_files/vp/009_1_vp_ave.fif',list(evokeds.values()), overwrite=True)